In [1]:
#!pip install -q -U watermark

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import sys, os

In [3]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



In [4]:
# Carregando DataFrames das edições do censo da educação superior.
censo_24 = 'censup/MICRODADOS_CADASTRO_CURSOS_2024.CSV'
censo_23 = 'censup/MICRODADOS_CADASTRO_CURSOS_2023.CSV'
censo_22 = 'censup/MICRODADOS_CADASTRO_CURSOS_2022.CSV'
censo_21 = 'censup/MICRODADOS_CADASTRO_CURSOS_2021.CSV'
censo_20 = 'censup/MICRODADOS_CADASTRO_CURSOS_2020.CSV' 
censo_19 = 'censup/MICRODADOS_CADASTRO_CURSOS_2019.CSV'
censo_18 = 'censup/MICRODADOS_CADASTRO_CURSOS_2018.CSV'
censo_17 = 'censup/MICRODADOS_CADASTRO_CURSOS_2017.CSV'
censo_16 = 'censup/MICRODADOS_CADASTRO_CURSOS_2016.CSV'
censo_15 = 'censup/MICRODADOS_CADASTRO_CURSOS_2015.CSV'
censo_14 = 'censup/MICRODADOS_CADASTRO_CURSOS_2014.CSV'
censo_13 = 'censup/MICRODADOS_CADASTRO_CURSOS_2013.CSV'
censo_12 = 'censup/MICRODADOS_CADASTRO_CURSOS_2012.CSV'
censo_11 = 'censup/MICRODADOS_CADASTRO_CURSOS_2011.CSV'
censo_10 = 'censup/MICRODADOS_CADASTRO_CURSOS_2010.CSV'


In [5]:
# Abrindo o arquivo como tipo binário 'rb' - 'read binary'
with open(censo_24, 'rb') as f1:
    raw_data_24 = f1.read(1000000)

In [6]:
# Descobrindo a codificação do dataset com o método .detect() da biblioteca chardet.
result_23 = chardet.detect(raw_data_24)
cod_23 = result_23['encoding']
conf_23 = result_23['confidence']
print(f'codificação detectada: {cod_23} (confiança: {conf_23:.2f}%)')

codificação detectada: ISO-8859-1 (confiança: 0.73%)


In [7]:
# Lista com a ordem desejada das colunas.
order_columns = ['NU_ANO_CENSO','CO_IES','NO_CURSO','CO_CURSO','TP_GRAU_ACADEMICO','IN_GRATUITO','TP_MODALIDADE_ENSINO','TP_NIVEL_ACADEMICO','NO_CINE_AREA_GERAL','CO_CINE_AREA_GERAL','CO_CINE_AREA_ESPECIFICA','NO_CINE_AREA_ESPECIFICA','CO_CINE_AREA_DETALHADA','NO_CINE_AREA_DETALHADA', 'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL', 'TP_DIMENSAO', 'TP_ORGANIZACAO_ACADEMICA', 'TP_REDE', 'TP_CATEGORIA_ADMINISTRATIVA',
                 'QT_CURSO', 'QT_VG_TOTAL', 'QT_VG_TOTAL_DIURNO', 'QT_VG_TOTAL_NOTURNO', 'QT_VG_TOTAL_EAD', 'QT_VG_NOVA', 'QT_VG_PROC_SELETIVO', 'QT_VG_REMANESC', 'QT_VG_PROG_ESPECIAL', 'QT_INSCRITO_TOTAL', 'QT_INSCRITO_TOTAL_DIURNO', 'QT_INSCRITO_TOTAL_NOTURNO', 'QT_INSCRITO_TOTAL_EAD', 'QT_INSC_VG_NOVA', 'QT_INSC_PROC_SELETIVO', 'QT_INSC_VG_REMANESC', 'QT_INSC_VG_PROG_ESPECIAL', 'QT_ING', 'QT_ING_FEM', 'QT_ING_MASC', 'QT_ING_DIURNO', 'QT_ING_NOTURNO', 'QT_ING_VG_NOVA', 'QT_ING_VESTIBULAR', 'QT_ING_ENEM',
                 'QT_ING_AVALIACAO_SERIADA', 'QT_ING_SELECAO_SIMPLIFICA', 'QT_ING_EGR', 'QT_ING_OUTRO_TIPO_SELECAO', 'QT_ING_PROC_SELETIVO', 'QT_ING_VG_REMANESC', 'QT_ING_VG_PROG_ESPECIAL', 'QT_ING_OUTRA_FORMA', 'QT_ING_0_17', 'QT_ING_18_24', 'QT_ING_25_29', 'QT_ING_30_34', 'QT_ING_35_39', 'QT_ING_40_49', 'QT_ING_50_59', 'QT_ING_60_MAIS', 'QT_ING_BRANCA', 'QT_ING_PRETA', 'QT_ING_PARDA', 'QT_ING_AMARELA', 'QT_ING_INDIGENA', 'QT_ING_CORND', 'QT_MAT', 'QT_MAT_FEM', 'QT_MAT_MASC', 'QT_MAT_DIURNO',
                 'QT_MAT_NOTURNO', 'QT_MAT_0_17', 'QT_MAT_18_24', 'QT_MAT_25_29', 'QT_MAT_30_34', 'QT_MAT_35_39', 'QT_MAT_40_49', 'QT_MAT_50_59', 'QT_MAT_60_MAIS', 'QT_MAT_BRANCA', 'QT_MAT_PRETA', 'QT_MAT_PARDA', 'QT_MAT_AMARELA', 'QT_MAT_INDIGENA', 'QT_MAT_CORND', 'QT_CONC', 'QT_CONC_FEM', 'QT_CONC_MASC', 'QT_CONC_DIURNO', 'QT_CONC_NOTURNO', 'QT_CONC_0_17', 'QT_CONC_18_24', 'QT_CONC_25_29', 'QT_CONC_30_34', 'QT_CONC_35_39', 'QT_CONC_40_49', 'QT_CONC_50_59', 'QT_CONC_60_MAIS', 'QT_CONC_BRANCA',
                 'QT_CONC_PRETA', 'QT_CONC_PRETA', 'QT_CONC_AMARELA', 'QT_CONC_INDIGENA', 'QT_CONC_CORND', 'QT_ING_NACBRAS', 'QT_ING_NACESTRANG', 'QT_MAT_NACBRAS', 'QT_MAT_NACESTRANG', 'QT_CONC_NACBRAS', 'QT_CONC_NACESTRANG', 'QT_ALUNO_DEFICIENTE', 'QT_ING_DEFICIENTE', 'QT_MAT_DEFICIENTE', 'QT_CONC_DEFICIENTE', 'QT_ING_FINANC', 'QT_ING_FINANC_REEMB','QT_ING_FIES', 'QT_ING_RPFIES', 'QT_ING_FINANC_REEMB_OUTROS', 'QT_ING_FINANC_NREEMB', 'QT_ING_PROUNII', 'QT_ING_PROUNIP', 'QT_ING_NRPFIES',
                 'QT_ING_FINANC_NREEMB_OUTROS', 'QT_MAT_FINANC', 'QT_MAT_FINANC_REEMB', 'QT_MAT_FIES', 'QT_MAT_RPFIES', 'QT_MAT_FINANC_REEMB_OUTROS', 'QT_MAT_FINANC_NREEMB', 'QT_MAT_PROUNII', 'QT_MAT_PROUNIP', 'QT_MAT_NRPFIES', 'QT_MAT_FINANC_NREEMB_OUTROS', 'QT_CONC_FINANC', 'QT_CONC_FINANC_REEMB', 'QT_CONC_FIES', 'QT_CONC_RPFIES', 'QT_CONC_FINANC_REEMB_OUTROS', 'QT_CONC_FINANC_NREEMB', 'QT_CONC_PROUNII', 'QT_CONC_PROUNIP', 'QT_CONC_NRPFIES', 'QT_CONC_FINANC_NREEMB_OUTROS', 'QT_ING_RESERVA_VAGA',
                 'QT_ING_RVREDEPUBLICA', 'QT_ING_RVPDEF', 'QT_ING_RVSOCIAL_RF', 'QT_ING_RVOUTROS', 'QT_MAT_RESERVA_VAGA', 'QT_MAT_RVREDEPUBLICA', 'QT_MAT_RVPDEF', 'QT_MAT_RVSOCIAL_RF', 'QT_MAT_RVSOCIAL_RF', 'QT_CONC_RESERVA_VAGA', 'QT_CONC_RVREDEPUBLICA', 'QT_CONC_RVSOCIAL_RF', 'QT_CONC_RVOUTROS', 'QT_SIT_TRANCADA', 'QT_SIT_DESVINCULADO', 'QT_SIT_TRANSFERIDO', 'QT_SIT_FALECIDO', 'QT_ING_PROCESCPUBLICA', 'QT_ING_PROCESCPRIVADA',
                 'QT_ING_PROCNAOINFORMADA','QT_MAT_PROCESCPUBLICA', 'QT_MAT_PROCESCPRIVADA', 'QT_MAT_PROCNAOINFORMADA', 'QT_CONC_PROCESCPUBLICA', 'QT_CONC_PROCESCPRIVADA', 'QT_CONC_PROCNAOINFORMADA', 'QT_PARFOR', 'QT_ING_PARFOR', 'QT_MAT_PARFOR', 'QT_CONC_PARFOR', 'QT_APOIO_SOCIAL', 'QT_ING_APOIO_SOCIAL', 'QT_MAT_APOIO_SOCIAL', 'QT_CONC_APOIO_SOCIAL', 'QT_ATIV_EXTRACURRICULAR', 'QT_ING_ATIV_EXTRACURRICULAR', 'QT_MAT_ATIV_EXTRACURRICULAR', 'QT_CONC_ATIV_EXTRACURRICULAR', 'QT_MOB_ACADEMICA', 'QT_ING_MOB_ACADEMICA',
                 'QT_MAT_MOB_ACADEMICA', 'QT_CONC_MOB_ACADEMICA']

In [8]:
# Dicionanário com os tipos de dados adequados por variável por coluna.
mapeamento_dtypes = {'NU_ANO_CENSO':'Int16','CO_IES':'category','NO_CURSO':'category','CO_CURSO':'category','TP_GRAU_ACADEMICO':'Int8','IN_GRATUITO':'Int8','TP_MODALIDADE_ENSINO':'Int8','TP_NIVEL_ACADEMICO':'Int8','NO_CINE_AREA_GERAL':'category','CO_CINE_AREA_GERAL':'category','CO_CINE_AREA_ESPECIFICA':'category','NO_CINE_AREA_ESPECIFICA':'category','CO_CINE_AREA_DETALHADA':'category','NO_CINE_AREA_DETALHADA':'category', 'NO_REGIAO':'category', 'CO_REGIAO':'category', 'NO_UF':'category', 'SG_UF':'category', 'CO_UF':'category', 'NO_MUNICIPIO':'category', 'CO_MUNICIPIO':'category', 'IN_CAPITAL':'Int8', 'TP_DIMENSAO':'Int8', 'TP_ORGANIZACAO_ACADEMICA':'Int8', 'TP_REDE':'Int8', 'TP_CATEGORIA_ADMINISTRATIVA': 'Int8',
                 'QT_CURSO':'Int32', 'QT_VG_TOTAL':'Int32', 'QT_VG_TOTAL_DIURNO':'Int32', 'QT_VG_TOTAL_NOTURNO':'Int32', 'QT_VG_TOTAL_EAD':'Int32', 'QT_VG_NOVA':'Int32', 'QT_VG_PROC_SELETIVO':'Int32', 'QT_VG_REMANESC':'Int32', 'QT_VG_PROG_ESPECIAL':'Int32', 'QT_INSCRITO_TOTAL':'Int32', 'QT_INSCRITO_TOTAL_DIURNO':'Int32', 'QT_INSCRITO_TOTAL_NOTURNO':'Int32', 'QT_INSCRITO_TOTAL_EAD':'Int32', 'QT_INSC_VG_NOVA':'Int32', 'QT_INSC_PROC_SELETIVO':'Int32', 'QT_INSC_VG_REMANESC':'Int32', 'QT_INSC_VG_PROG_ESPECIAL':'Int32', 'QT_ING':'Int32', 'QT_ING_FEM':'Int32', 'QT_ING_MASC':'Int32', 'QT_ING_DIURNO':'Int32', 'QT_ING_NOTURNO':'Int32', 'QT_ING_VG_NOVA':'Int32', 'QT_ING_VESTIBULAR':'Int32', 'QT_ING_ENEM':'Int32',
                 'QT_ING_AVALIACAO_SERIADA':'Int32', 'QT_ING_SELECAO_SIMPLIFICA':'Int32', 'QT_ING_EGR':'Int32', 'QT_ING_OUTRO_TIPO_SELECAO':'Int32', 'QT_ING_PROC_SELETIVO':'Int32', 'QT_ING_VG_REMANESC':'Int32', 'QT_ING_VG_PROG_ESPECIAL':'Int32', 'QT_ING_OUTRA_FORMA':'Int32', 'QT_ING_0_17':'Int32', 'QT_ING_18_24':'Int32', 'QT_ING_25_29':'Int32', 'QT_ING_30_34':'Int32', 'QT_ING_35_39':'Int32', 'QT_ING_40_49':'Int32', 'QT_ING_50_59':'Int32', 'QT_ING_60_MAIS':'Int32', 'QT_ING_BRANCA':'Int32', 'QT_ING_PRETA':'Int32', 'QT_ING_PARDA':'Int32', 'QT_ING_AMARELA':'Int32', 'QT_ING_INDIGENA':'Int32', 'QT_ING_CORND':'Int32', 'QT_MAT':'Int32', 'QT_MAT_FEM':'Int32', 'QT_MAT_MASC':'Int32', 'QT_MAT_DIURNO':'Int32',
                 'QT_MAT_NOTURNO':'Int32', 'QT_MAT_0_17':'Int32', 'QT_MAT_18_24':'Int32', 'QT_MAT_25_29':'Int32', 'QT_MAT_30_34':'Int32', 'QT_MAT_35_39':'Int32', 'QT_MAT_40_49':'Int32', 'QT_MAT_50_59':'Int32', 'QT_MAT_60_MAIS':'Int32', 'QT_MAT_BRANCA':'Int32', 'QT_MAT_PRETA':'Int32', 'QT_MAT_PARDA':'Int32', 'QT_MAT_AMARELA':'Int32', 'QT_MAT_INDIGENA':'Int32', 'QT_MAT_CORND':'Int32', 'QT_CONC':'Int32', 'QT_CONC_FEM':'Int32', 'QT_CONC_MASC':'Int32', 'QT_CONC_DIURNO':'Int32', 'QT_CONC_NOTURNO':'Int32', 'QT_CONC_0_17':'Int32', 'QT_CONC_18_24':'Int32', 'QT_CONC_25_29':'Int32', 'QT_CONC_30_34':'Int32', 'QT_CONC_35_39':'Int32', 'QT_CONC_40_49':'Int32', 'QT_CONC_50_59':'Int32', 'QT_CONC_60_MAIS':'Int32', 'QT_CONC_BRANCA':'Int32',
                 'QT_CONC_PRETA':'Int32', 'QT_CONC_PRETA':'Int32', 'QT_CONC_AMARELA':'Int32', 'QT_CONC_INDIGENA':'Int32', 'QT_CONC_CORND':'Int32', 'QT_ING_NACBRAS':'Int32', 'QT_ING_NACESTRANG':'Int32', 'QT_MAT_NACBRAS':'Int32', 'QT_MAT_NACESTRANG':'Int32', 'QT_CONC_NACBRAS':'Int32', 'QT_CONC_NACESTRANG':'Int32', 'QT_ALUNO_DEFICIENTE':'Int32', 'QT_ING_DEFICIENTE':'Int32', 'QT_MAT_DEFICIENTE':'Int32', 'QT_CONC_DEFICIENTE':'Int32', 'QT_ING_FINANC':'Int32', 'QT_ING_FINANC_REEMB':'Int32','QT_ING_FIES':'Int32', 'QT_ING_RPFIES':'Int32', 'QT_ING_FINANC_REEMB_OUTROS':'Int32', 'QT_ING_FINANC_NREEMB':'Int32', 'QT_ING_PROUNII':'Int32', 'QT_ING_PROUNIP':'Int32', 'QT_ING_NRPFIES':'Int32',
                 'QT_ING_FINANC_NREEMB_OUTROS':'Int32', 'QT_MAT_FINANC':'Int32', 'QT_MAT_FINANC_REEMB':'Int32', 'QT_MAT_FIES':'Int32', 'QT_MAT_RPFIES':'Int32', 'QT_MAT_FINANC_REEMB_OUTROS':'Int32', 'QT_MAT_FINANC_NREEMB':'Int32', 'QT_MAT_PROUNII':'Int32', 'QT_MAT_PROUNIP':'Int32', 'QT_MAT_NRPFIES':'Int32', 'QT_MAT_FINANC_NREEMB_OUTROS':'Int32', 'QT_CONC_FINANC':'Int32', 'QT_CONC_FINANC_REEMB':'Int32', 'QT_CONC_FIES':'Int32', 'QT_CONC_RPFIES':'Int32', 'QT_CONC_FINANC_REEMB_OUTROS':'Int32', 'QT_CONC_FINANC_NREEMB':'Int32', 'QT_CONC_PROUNII':'Int32', 'QT_CONC_PROUNIP':'Int32', 'QT_CONC_NRPFIES':'Int32', 'QT_CONC_FINANC_NREEMB_OUTROS':'Int32', 'QT_ING_RESERVA_VAGA':'Int32',
                 'QT_ING_RVREDEPUBLICA':'Int32', 'QT_ING_RVPDEF':'Int32', 'QT_ING_RVSOCIAL_RF':'Int32', 'QT_ING_RVOUTROS':'Int32', 'QT_MAT_RESERVA_VAGA':'Int32', 'QT_MAT_RVREDEPUBLICA':'Int32', 'QT_MAT_RVPDEF':'Int32', 'QT_MAT_RVSOCIAL_RF':'Int32', 'QT_MAT_RVSOCIAL_RF':'Int32', 'QT_CONC_RESERVA_VAGA':'Int32', 'QT_CONC_RVREDEPUBLICA':'Int32', 'QT_CONC_RVSOCIAL_RF':'Int32', 'QT_CONC_RVOUTROS':'Int32', 'QT_SIT_TRANCADA':'Int32', 'QT_SIT_DESVINCULADO':'Int32', 'QT_SIT_TRANSFERIDO':'Int32', 'QT_SIT_FALECIDO':'Int32', 'QT_ING_PROCESCPUBLICA':'Int32', 'QT_ING_PROCESCPRIVADA':'Int32',
                 'QT_ING_PROCNAOINFORMADA':'Int32','QT_MAT_PROCESCPUBLICA':'Int32', 'QT_MAT_PROCESCPRIVADA':'Int32', 'QT_MAT_PROCNAOINFORMADA':'Int32', 'QT_CONC_PROCESCPUBLICA':'Int32', 'QT_CONC_PROCESCPRIVADA':'Int32', 'QT_CONC_PROCNAOINFORMADA':'Int32', 'QT_PARFOR':'Int32', 'QT_ING_PARFOR':'Int32', 'QT_MAT_PARFOR':'Int32', 'QT_CONC_PARFOR':'Int32', 'QT_APOIO_SOCIAL':'Int32', 'QT_ING_APOIO_SOCIAL':'Int32', 'QT_MAT_APOIO_SOCIAL':'Int32', 'QT_CONC_APOIO_SOCIAL':'Int32', 'QT_ATIV_EXTRACURRICULAR':'Int32', 'QT_ING_ATIV_EXTRACURRICULAR':'Int32', 'QT_MAT_ATIV_EXTRACURRICULAR':'Int32', 'QT_CONC_ATIV_EXTRACURRICULAR':'Int32', 'QT_MOB_ACADEMICA':'Int32', 'QT_ING_MOB_ACADEMICA':'Int32',
                 'QT_MAT_MOB_ACADEMICA':'Int32', 'QT_CONC_MOB_ACADEMICA':'Int32'}

In [9]:
# Carregando o DataFrame em uma variável.
df_24 = pd.read_csv(censo_24, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_24.shape

(720349, 192)

In [10]:
df_24['CO_CURSO'].duplicated().any()

True

In [14]:
df_23 = pd.read_csv(censo_23, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_23.shape

(671610, 192)

In [15]:
df_23['CO_CURSO'].duplicated().any()

True

In [16]:
df_22 = pd.read_csv(censo_22, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_22.shape

(573019, 192)

In [17]:
df_22['CO_CURSO'].duplicated().any()

True

In [18]:
df_21 = pd.read_csv(censo_21, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_21.shape

(444786, 192)

In [19]:
df_21['CO_CURSO'].duplicated().any()

True

In [20]:
df_20 = pd.read_csv(censo_20, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_20.shape

(344691, 192)

In [21]:
df_20['CO_CURSO'].duplicated().any()

True

In [22]:
df_19 = pd.read_csv(censo_19, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_19.shape

(253139, 192)

In [23]:
df_19['CO_CURSO'].duplicated().any()

True

In [24]:
df_18 = pd.read_csv(censo_18, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_18.shape

(182892, 192)

In [25]:
df_18['CO_CURSO'].duplicated().any()

True

In [26]:
df_17 = pd.read_csv(censo_17, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_17.shape

(119798, 192)

In [27]:
df_17['CO_CURSO'].duplicated().any()

True

In [28]:
df_16 = pd.read_csv(censo_16, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_16.shape

(92866, 192)

In [29]:
df_16['CO_CURSO'].duplicated().any()

True

In [30]:
df_15 = pd.read_csv(censo_15, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_15.shape

(81156, 192)

In [31]:
df_15['CO_CURSO'].duplicated().any()

True

In [32]:
df_14 = pd.read_csv(censo_14, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_14.shape

(73569, 192)

In [33]:
df_14['CO_CURSO'].duplicated().any()

True

In [34]:
df_13 = pd.read_csv(censo_13, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_13.shape

(66126, 192)

In [35]:
df_13['CO_CURSO'].duplicated().any()

True

In [36]:
df_12 = pd.read_csv(censo_12, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_12.shape

(63855, 192)

In [37]:
df_12['CO_CURSO'].duplicated().any()

True

In [38]:
df_11 = pd.read_csv(censo_11, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_11.shape

(57737, 192)

In [39]:
df_11['CO_CURSO'].duplicated().any()

True

In [40]:
df_10 = pd.read_csv(censo_10, sep=';', encoding=cod_23, low_memory=False, decimal=',', usecols=order_columns, dtype=mapeamento_dtypes)
df_10.shape

(54194, 192)

In [41]:
df_10['CO_CURSO'].duplicated().any()

True

In [110]:
# Alterando o parâmetro máximo de visualizações de colunas
pd.set_option('display.max_columns', 202)

In [111]:
# Alterando o parâmetro máximo de visualização de linhas
pd.set_option('display.max_rows', 500)

### Identificando as colunas que são comuns a todos os DataFrames e, em seguida, manter apenas essas colunas em cada um deles.
Este procedimento garante que, ao concatenar, você não serão mostradas colunas com valores 'NaN'.
### 1.1 Encontrando as colunas comuns

In [44]:
# Carregando os dfs em uma lista
dfs = [df_10, df_11, df_12, df_13, df_14, df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22, df_23, df_24]
len(dfs)

15

In [45]:
# 1.2 Pegando convertendo a lista de colunas de cada df para o tipo set()
colunas_10 = set(dfs[0].columns)
colunas_11 = set(dfs[1].columns)
colunas_12 = set(dfs[2].columns)
colunas_13 = set(dfs[3].columns)
colunas_14 = set(dfs[4].columns)
colunas_15 = set(dfs[5].columns)
colunas_16 = set(dfs[6].columns)
colunas_17 = set(dfs[7].columns)
colunas_18 = set(dfs[8].columns)
colunas_19 = set(dfs[9].columns)
colunas_20 = set(dfs[10].columns)
colunas_21 = set(dfs[11].columns)
colunas_22 = set(dfs[12].columns)
colunas_23 = set(dfs[13].columns)
colunas_24 = set(dfs[14].columns)

**Encontrandoa interseção de todas as colunas**

In [46]:
# O resultado será um conjunto de colunas presentes em todos os DataFrames
colunas_comuns = colunas_10.intersection(colunas_11).intersection(colunas_12).intersection(colunas_13).intersection(colunas_14).intersection(colunas_15).intersection(colunas_16).intersection(colunas_17).intersection(colunas_18).intersection(colunas_19).intersection(colunas_20).intersection(colunas_21).intersection(colunas_22).intersection(colunas_23).intersection(colunas_24)

In [47]:
# Converte o conjunto de volta para o formato de lista
colunas_comuns_lista = list(colunas_comuns)

In [48]:
print(f"Número total de colunas em 2010: {len(colunas_10)}")
print(f"Número total de colunas em 2011: {len(colunas_11)}")
print(f"Número total de colunas em 2012: {len(colunas_12)}")
print(f"Número total de colunas em 2013: {len(colunas_13)}")
print(f"Número total de colunas em 2014: {len(colunas_14)}")
print(f"Número total de colunas em 2015: {len(colunas_15)}")
print(f"Número total de colunas em 2016: {len(colunas_16)}")
print(f"Número total de colunas em 2017: {len(colunas_17)}")
print(f"Número total de colunas em 2018: {len(colunas_18)}")
print(f"Número total de colunas em 2019: {len(colunas_19)}")
print(f"Número total de colunas em 2020: {len(colunas_20)}")
print(f"Número total de colunas em 2021: {len(colunas_21)}")
print(f"Número total de colunas em 2022: {len(colunas_22)}")
print(f"Número total de colunas em 2023: {len(colunas_23)}")
print(f"Número total de colunas em 2024: {len(colunas_24)}")
print(f"Número de colunas comuns {len(colunas_comuns)}")

Número total de colunas em 2010: 192
Número total de colunas em 2011: 192
Número total de colunas em 2012: 192
Número total de colunas em 2013: 192
Número total de colunas em 2014: 192
Número total de colunas em 2015: 192
Número total de colunas em 2016: 192
Número total de colunas em 2017: 192
Número total de colunas em 2018: 192
Número total de colunas em 2019: 192
Número total de colunas em 2020: 192
Número total de colunas em 2021: 192
Número total de colunas em 2022: 192
Número total de colunas em 2023: 192
Número total de colunas em 2024: 192
Número de colunas comuns 192


# Criando uma lista com as colunas originais.
lista_colunas_originais = [colunas_10, colunas_11, colunas_12, colunas_13, colunas_14,
                           colunas_15, colunas_16, colunas_17, colunas_18, colunas_19,
                           colunas_20, colunas_21, colunas_22, colunas_23, colunas_24]

# Criando lista com os nomes dos dfs.
nomes_dfs = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

for colunas_originais, df_nome in zip(lista_colunas_originais, nomes_dfs):
    # Calcula as colunas que estavam no original, mas NÃO estão nas colunas comuns
    colunas_deletadas = colunas_originais - colunas_comuns

    if len(colunas_deletadas) > 0:
        print(f"[{df_nome}]: Foram DELETADAS {len(colunas_deletadas)} colunas.")
        print("     Colunas deletadas: \n  ", '\n    '.join(colunas_deletadas))
    else:
        print(F"[{df_nome}]: Nenhuma coluna foi deletada neste DataFrame.")
    print("-" * 30)

In [49]:
# Iterando sobre a lista de DataFrames
dfs_padronizados = []
for df_ano in dfs:
    # Selecionar apenas as colunas comuns para cada DataFrame
    df_padronizado = df_ano[colunas_comuns_lista]
    dfs_padronizados.append(df_padronizado)

# Concatenando os DataFrames padronizados em um único DataFrame final
df_completo_padronizado = pd.concat(dfs_padronizados, ignore_index=True)

In [50]:
df_completo_padronizado.shape

(3799787, 192)

In [51]:
df_completo_padronizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799787 entries, 0 to 3799786
Columns: 192 entries, QT_MAT_FIES to QT_CONC_PROUNIP
dtypes: Int16(1), Int32(166), Int8(9), category(4), object(12)
memory usage: 3.4+ GB


**Alterando o tipo de dado por coluna (Após concatenação de df)**

In [52]:
# Alterando o tipo 'object' de dados com loop for
colunas = df_completo_padronizado.columns

# 1. Otimização para Códigos Categóricos (Tipos de informação)
for col in colunas:
    # Colunas que são geralmente códigos, indicadores ou categorias com poucos valores
    if col.startswith('TP_') or col.startswith('IN_'):
        # Usar Int8 para códigos pequenos (como 0 até 9) ou indicadores (0/1)
        df_completo_padronizado[col] = df_completo_padronizado[col].astype('Int8')

# 2. Otimização para Textos (Nomes e Descrições)
for col in colunas:
    # Colunas de códigos (CO_) ou nomes repetitivos (NO_)
    if col.startswith('CO_') or col.startswith('NO_') or col.startswith('SG_'):
        # 'category' é o tipo mais eficiênte para strings repetitivas
        df_completo_padronizado[col] = df_completo_padronizado[col].astype('category')

In [53]:
# Validação da alteração no tipo de dados.
df_completo_padronizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799787 entries, 0 to 3799786
Columns: 192 entries, QT_MAT_FIES to QT_CONC_PROUNIP
dtypes: Int16(1), Int32(166), Int8(9), category(16)
memory usage: 3.1 GB


In [54]:
# Salvando uma cópia com as colunas ordenadas.
df_full_order = df_completo_padronizado[order_columns].copy()

In [55]:
# Validando as dimensões
df_full_order.shape

(3799787, 194)

In [56]:
# Vasualização das primeiras linhas
df_full_order.head(2)

,NU_ANO_CENSO,CO_IES,NO_CURSO,CO_CURSO,TP_GRAU_ACADEMICO,IN_GRATUITO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,NO_CINE_AREA_GERAL,CO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,TP_ORGANIZACAO_ACADEMICA,TP_REDE,TP_CATEGORIA_ADMINISTRATIVA,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PRETA,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVSOCIAL_RF,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
0,2010,699,ENGENHARIA DE ALIMENTOS,116776,1,1,1,1,"Engenharia, produção e construção",07,072,Produção e processamento,0721,Processamento de alimentos,Norte,1,Rondônia,RO,11,Ariquemes,1100023,0,1,1,1,1,1,50,50,0,0,0,0,0,0,252,252,0,0,0,0,0,0,44,29,15,44,0,0,3,4,0,0,0,0,0,0,0,37,4,32,3,3,2,0,0,0,3,0,27,1,0,13,58,41,17,58,0,2,46,5,3,2,0,0,0,16,2,35,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44,0,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,27,1,0,0,0,44,0,0,58,0,0,0,0,0,0,0,7,0,7,0,0,0,0,0,0,0,0,0
1,2010,699,PEDAGOGIA,116782,2,1,1,1,Educação,01,011,Educação,0113,Formação de professores sem áreas específicas,Norte,1,Rondônia,RO,11,Ariquemes,1100023,0,1,1,1,1,

In [57]:
# Criando um df apenas com os cursos da modalidade EaD
#df_censo_ead = df_completo_padronizado[df_completo_padronizado['TP_MODALIDADE_ENSINO'] == 2].copy()

In [58]:
#df_censo_ead.shape

### Salvando uma cópia apenas com Cursos EaD

In [59]:
# df_censo_ead.to_csv('panorama_ead.csv', sep=";", encoding=cod_23, index=False)

In [60]:
# Substuindo valores da variável 'TP_MODALIDADE_ENSINO'
# df_censo_ead['TP_MODALIDADE_ENSINO'] = df_censo_ead['TP_MODALIDADE_ENSINO'].replace(2, 'EaD')

In [61]:
# df_censo_ead['NU_ANO_CENSO'] = pd.to_datetime(df_censo_ead['NU_ANO_CENSO'], format='%Y')

In [62]:
# Alterando o tipo da coluna 'NU_ANO_CENSO' para o df completo.
# df_completo_padronizado['NU_ANO_CENSO'] = pd.to_datetime(df_completo_padronizado['NU_ANO_CENSO'], format='%Y')

### Tratamento de Valores Nulos das Variaveis Numéricas

In [63]:
# Selecionando apenas colunas com valores numéricos
colunas_quantidade = [col for col in df_full_order.columns if col.startswith('QT_')]
len(colunas_quantidade)

168

In [64]:
# Preechendo os valores nulos (NaN) para evitar erros nas operações matemáricas, usando 0 nas colunas de contagem
for col in colunas_quantidade:
    df_full_order[col] = df_full_order[col].fillna(0)

# Solução alternativa usando .loc
# df_full_order.loc[:,colunas_quantidade] = df_full_order[colunas_quantidade].fillna(0)

### 1. Métricas de Conclusão e Evasão
Métricas importantes para entender a eficiência e retenção dos cursos.

### Qual foi o total de ingressantes no EaD em cada ano?

In [65]:
# df_censo_ead.groupby('NU_ANO_CENSO')['QT_ING'].sum().sort_values(ascending=False)

In [66]:
# Matrícula
# df_censo_ead.groupby('NU_ANO_CENSO')['QT_MAT'].sum().sort_values(ascending=False)

In [67]:
# Concluintes
# df_censo_ead.groupby('NU_ANO_CENSO')['QT_CONC'].sum().sort_values(ascending=False)

In [68]:
# Ingresstentes por Categoria Administrativa
# df_censo_ead.groupby(['NU_ANO_CENSO', 'TP_REDE'])['QT_ING'].sum()

### Qual foi a quantidade de Ingressantes com mais de 50 anos em 2024?

In [69]:
# Quantidade de ingressantes por censo
df_completo_padronizado.groupby('NU_ANO_CENSO')[['QT_ING_60_MAIS', 'QT_ING_50_59' ]].sum()

In [696]:
# Quantidade de ingressantes por censo
df_full_order.groupby(['NU_ANO_CENSO','TP_MODALIDADE_ENSINO', 'TP_GRAU_ACADEMICO'])[['QT_ING', 'QT_MAT', 'QT_CONC' ]].sum()

QT_ING   QT_MAT  QT_CONC
NU_ANO_CENSO TP_MODALIDADE_ENSINO TP_GRAU_ACADEMICO                           
2010         1                    1                  1227657  3958544   556336
                                  2                   298390   928748   161354
                                  3                   268040   545844   111596
                                  4                     7814    15984        0
                                  5                    14536    27693     6520
             2                    1                   112346   268173    21555
                                  2                   154137   426241    71952
                                  3                   113845   235765    51046
                                  5                       57      741      303
2011         1                    1                  1316546  4196423   582995
                                  2                   285750   926780   160883
                                  3                   303053   606564   121283
                                  4                     9749    16995        0
                                  5                    12680    25716     5921
             2                    1                   122435   299408    24976
                                  2                   168962   429549    77224
                                  3                   140200   263970    49352
                                  5                       34      135       77
2012         1                    1                  1541750  4344030   592080
                                  2                   304888   916593   148229
                                  3                   347370   640683   135782
                                  4                    10448    22532        0
                                  5                     9621    20288     5644
             2                    1                   161954   359663    45406
                                  2                   186199   449966    75663
                                  3                   194480   304221    53253
                                  5                       63      108       12
2013         1                    1                  1584909  4551108   552060
                                  2                   301264   922981   140036
                                  3                   327697   654569   137842
                                  4                    13675    23747        0
                                  5                     6848    16919     3776
             2                    1                   153363   361202    42635
                                  2                   167973   451193    61317
                                  3                   194069   341177    57120
                                  5                        5       68       26
2014         1                    1                  1743056  4892907   560879
                                  2                   296575   925942   142376
                                  3                   323379   645125   134049
                                  4                    20100    22197        0
                                  5                     3648    11718     3424
             2                    1                   209272   416507    43025
                                  2                   271872   540693    74683
                                  3                   246594   384642    72080
                                  5                       14       34        4
2015         1                    1                  1645377  5080073   620513
                                  2                   276575   906930   152382
                                  3                   282184   617468   143468
                                  4                    21527    29074        0
                         

In [70]:
# Quantidade de Ingressantes com 60 anos ou mais por modalidade de ensino.
#df_completo_padronizado.groupby(['NU_ANO_CENSO', 'TP_MODALIDADE_ENSINO'])[['QT_ING_60_MAIS', 'QT_ING_50_59']].sum()

In [71]:
# Quantidade de concluintes com + 60anos por censo
#df_completo_padronizado.groupby('NU_ANO_CENSO')[['QT_CONC_60_MAIS', 'QT_CONC_50_59']].sum()

In [72]:
# Quantidade de Concluintes entre 50 e 59 anos por ano e modalidade.
df_completo_padronizado.groupby(['NU_ANO_CENSO', 'TP_MODALIDADE_ENSINO'])[['QT_CONC_60_MAIS', 'QT_CONC_50_59']].sum()

### Análise Exploratória de Dados - EDA
**Análise de Valores Ausentes**

In [73]:
# Primeiras estatísticas
df_full_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799787 entries, 0 to 3799786
Columns: 194 entries, NU_ANO_CENSO to QT_CONC_MOB_ACADEMICA
dtypes: Int16(1), Int32(168), Int8(9), category(16)
memory usage: 3.1 GB


###  Cálculo do Percentual de Valores Ausentes (Geral)

In [686]:
def percent_values_miss(df):
    # Cálculo do total de celulas do df
    total_cells = np.prod(df.shape)
    # Cálculo do nº de valores ausentes por coluna
    columns_miss = df.isnull().sum()
    # Cálculo do total de valores ausentes
    total_missing = columns_miss.sum()
    # Cálculo do percentual de valores ausentes
    print("O dataset tem", round(((total_missing / total_cells) * 100),2), "%", "de valores ausentes.")

In [687]:
percent_values_miss(df_full_order)

O dataset tem 0.0 % de valores ausentes.


### Cálculo de Valores Ausentes por Coluna

In [645]:
def percent_colum_val_miss(df):
    # Total de valores ausentes em cada coluna do DataFrame
    miss_values = df.isnull().sum()
    # Percentual de valores ausentes
    p_miss_values = 100 * miss_values / len(df)
    # Tipo de dados das colunas com valores ausentes
    miss_values_types = df.dtypes
    # Criando um tabela com os resultados
    miss_table = pd.concat([miss_values, p_miss_values, miss_values_types], axis=1)

    # Renomeando as colunas
    miss_values_table = miss_table.rename(columns = {0: 'Valores Ausentes', 1: '% de Valores Ausentes', 2: 'Dtypes'})
    # Classificar a tabela por porcentagem de valores ausentes de forma decrescente e remover colunas sem valores ausentes
    miss_values_table = miss_values_table[miss_values_table.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)

    # Print
    print("O dataset tem " + str(df.shape[1]) + " colunas.\n" "Encontrando: " + str(miss_values_table.shape[0]) + " colunas que têm valores ausentes.")
    if miss_values_table.shape[0] == 0:
        return

    # Retorna o da DataFrame com informações ausentes
    return miss_values_table

In [688]:
# Criando uma tabela com valores ausentes
df_missing = percent_colum_val_miss(df_full_order)

O dataset tem 194 colunas.
Encontrando: 0 colunas que têm valores ausentes.


**Visualização da Tabela**

In [684]:
df_missing

**Filtragem por nulos**

In [681]:
df_full_order['CO_MUNICIPIO'].unique()

['1100023', '1100049', '1100064', '1100106', '1100114', ..., '4126108', '4303673', '4201109', '4214300', '4216404']
Length: 3797
Categories (3797, object): ['0', '1100015', '1100023', '1100049', ..., '5222005', '5222054', '5300108', 'ND*']

In [631]:
# 1. Cria uma Série Booleana (True onde o valor é NA)
mask_na = df_full_order['NO_MUNICIPIO'].isna()

**Método correto para adicionar uma nova string em uma coluna do tipo 'category'**

In [680]:
# Adicionar a nova string 'ND*' á lista de categorias válidas da coluna 'NO_REGIAO'
df_full_order['CO_MUNICIPIO'] = df_full_order['CO_MUNICIPIO'].cat.add_categories('ND*')

In [682]:
# Preencher os nulos com a nova categoria, atribuindo o resultado de volta na caluna
df_full_order['CO_MUNICIPIO'] = df_full_order['CO_MUNICIPIO'].fillna('ND*')

In [632]:
# 2. Aplica a máscara para selecionar apenas as linhas onde valor é True
registros_na = df_full_order[mask_na]

**Visualizando os registros de uma valor específico**

In [633]:
registros_na['NO_CURSO'].value_counts()


NO_CURSO
PEDAGOGIA                            1820
Pedagogia                            1427
ADMINISTRAÇÃO                        1401
Administração                        1166
GESTÃO DE RECURSOS HUMANOS           1015
                                     ... 
GESTÃO DE COMUNICAÇÃO EMPRESARIAL       0
GESTÃO DE COMUNICAÇÕES MILITARES        0
GESTÃO DE COMÉRCIO                      0
GESTÃO DE COMÉRCIO E DE SERVIÇOS        0
ABI - ANTROPOLOGIA                      0
Name: count, Length: 3912, dtype: int64

**Tentantiva de identificação do grau acadêmico dos registros não informados**

In [637]:
registros_na.sample()

,NU_ANO_CENSO,CO_IES,NO_CURSO,CO_CURSO,TP_GRAU_ACADEMICO,IN_GRATUITO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,NO_CINE_AREA_GERAL,CO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,TP_ORGANIZACAO_ACADEMICA,TP_REDE,TP_CATEGORIA_ADMINISTRATIVA,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PRETA,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVSOCIAL_RF,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
3091034,2024,1491,Geografia,1314212,2,0,2,1,Educação,01,011,Educação,0114,Formação de professores em áreas específicas (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,4,2,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,4,2,2,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [635]:
registros_na['TP_MODALIDADE_ENSINO'].unique()

<IntegerArray>
[2]
Length: 1, dtype: Int8

In [626]:
df_pb = registros_na[registros_na['NO_CINE_AREA_GERAL'] == 'Negócios, administração e direito']

In [627]:
n_cursos1 = pd.DataFrame(df_pb['NO_CURSO'].value_counts().reset_index())

**Abi - Educão Física**

É um modelo de curso onde o estudante ingressa em um cíclo básico comum, cursando disciplinas que integram a formação para licenciatura e bacharelado.

**Tratamento dos valores ausentes para os cursos ABI - Educação Física, para a variável 'TP_GRAU_ACADEMICO'**

Preechimento dos valores ausentes com o dígito 4 (Bacharelado e Licenciatura)

In [606]:
view = registros_na[registros_na['IN_GRATUITO'] == 0]
view['TP_CATEGORIA_ADMINISTRATIVA'].unique()

<IntegerArray>
[5, 4, 7]
Length: 3, dtype: Int8

In [607]:
# Cursos MATEMÁTICA - LICENCIATURA OU BACHARELADO (Área Básica de Ingresso)
df_abi_ma = df_full_order[df_full_order['NO_CURSO'] == 'ADMINISTRAÇÃO']
df_abi_ma.shape

(71442, 194)

In [690]:
df_abi_ma.sample()

,NU_ANO_CENSO,CO_IES,NO_CURSO,CO_CURSO,TP_GRAU_ACADEMICO,IN_GRATUITO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,NO_CINE_AREA_GERAL,CO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,TP_ORGANIZACAO_ACADEMICA,TP_REDE,TP_CATEGORIA_ADMINISTRATIVA,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PRETA,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVSOCIAL_RF,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
31994,2010,322,ADMINISTRAÇÃO,100326,1,0,2,1,"Negócios, administração e direito",04,041,Negócios e administração,0413,Gestão e administração,Nordeste,2,Ceará,CE,23,Limoeiro do Norte,2307601,0,2,1,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [579]:
df_abi_ma['IN_GRATUITO'].unique()

<IntegerArray>
[0, 1]
Length: 2, dtype: Int8

In [616]:
df_full_order['TP_REDE'].unique()

<IntegerArray>
[1, 2]
Length: 2, dtype: Int8

**Aplicação de Imputação Condicional pontual usando .loc[]**

In [548]:
#Preenchimento dos valores ausentes da coluna 'TP_GRAU_ACADEMICO' para os cursos  CIÊNCIAS SOCIAIS - LICENCIATURA OU BACHARELADO'
conditions = ((df_full_order['TP_GRAU_ACADEMICO'] == 0) & (df_full_order['TP_NIVEL_ACADEMICO'] == 2))

In [549]:
#Aplicando a condição usando o método .loc[]
df_full_order.loc[conditions, 'TP_GRAU_ACADEMICO'] = 5

**Aplicação de Imputação Condicional em bloco usando .loc[]**

In [516]:
# 1. Definir a condição de NULIDADE na variável 'TP_GRAU_ACADEMICO'
condition_isna = df_full_order['TP_GRAU_ACADEMICO'] == 0  #.isna()
# 2. Definindo a condição da variável 'NO_CURSO'
# - Converter a coluna para string
# - Converter a string para maiúscula
# - Verificar se a string começa com 'ABI - ' (padrão maiúculo)
condition_standart = (df_full_order['NO_CURSO'].astype(str).str.upper().str.startswith('ABI -'))

# 3. Combina as duas condições (NA e Padrão ABI)
conditions = (condition_isna & condition_standart)
# 4. Aplica a alteração (Imputação Condicional)
df_full_order.loc[conditions, 'TP_GRAU_ACADEMICO'] = 4

print(f"Total de registros corrigidos (NA preenchidos com 4): {conditions.sum()}")

Total de registros corrigidos (NA preenchidos com 4): 220


**Aplicação de Imputação Condicional em lote usando .isin()**

In [593]:
# 1. Definindo os valores da variável que será usada para validação.
valores_referencia = [1, 2, 3] # TP_CATEGORIA_ADMINISTRATIVA (PÚBLICAS)

# 2. Definindo a condição de NULIDADE no variável 'TP_REDE'
condicao_isna = df_full_order['TP_REDE'].isna()

# 3. Definir a condição da coluna secundária:
# - Usando o método .isin() para verificar se o valor da linha está em 'valores_referencia'
condicao_secundaria = df_full_order['TP_CATEGORIA_ADMINISTRATIVA'].isin(valores_referencia)

# 4. Combinando as duas condições (NA e [1,2,3])
conditions = (condicao_isna & condicao_secundaria)

# 5. Aplicar a alteração (Imputação Condicional)
valor = 1
df_full_order.loc[conditions, 'TP_REDE'] = valor

print(f"Total de registros atribuidos: {conditions.sum()}")

Total de registros atribuidos: 12574


In [615]:
# 1. Definindo os valores da variável que será usada para validação.
valores_referencia = [4,5,6,7,8,9] # TP_CATEGORIA_ADMINISTRATIVA (PÚBLICAS)

# 2. Definindo a condição de NULIDADE no variável 'TP_REDE'
condicao_isna = df_full_order['TP_REDE'] == 0

# 3. Definir a condição da coluna secundária:
# - Usando o método .isin() para verificar se o valor da linha está em 'valores_referencia'
condicao_secundaria = df_full_order['TP_CATEGORIA_ADMINISTRATIVA'].isin(valores_referencia)

# 4. Combinando as duas condições (NA e [1,2,3])
conditions = (condicao_isna & condicao_secundaria)

# 5. Aplicar a alteração (Imputação Condicional)
valor = 2
df_full_order.loc[conditions, 'TP_REDE'] = valor

print(f"Total de registros atribuidos: {conditions.sum()}")

Total de registros atribuidos: 52291


In [97]:
#registros_na['NO_CINE_AREA_ESPECIFICA'].value_counts()

In [88]:
#6. Verificando a quantidade dos valores único na coluna 'TP_NIVEL_ACADEMICO'.
registros_na['TP_GRAU_ACADEMICO'].value_counts()

Series([], Name: count, dtype: Int64)

In [125]:
#5. Confirmando os valores únicos da caluna.
registros_na['TP_NIVEL_ACADEMICO'].unique()

<IntegerArray>
[2, 1]
Length: 2, dtype: Int8

**Imputação Condicional no Pandas**

O .loc[ ] permite filtrar o DataFrame com base em condições booleanas e, em seguida, atribuir um novo valor às linhas que atendam essa condição.

In [ ]:
# 1. Definindo a Condição
condition = (df_completo_padronizado['TP_GRAU_ACADEMICO'].isna() & (df_completo_padronizado['TP_NIVEL_ACADEMICO'] == 2))

In [ ]:
# 2. Aplicando a atribuição usando .loc[]
df_completo_padronizado.loc[condition, 'TP_GRAU_ACADEMICO'] = 5

In [ ]:
df_completo_padronizado['TP_GRAU_ACADEMICO'].value_counts()

### Criando Novas Bases'.csv' Por Edição do Censo da Educação Superir

In [ ]:
#CENSO 2010
df_2010 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2010]

In [ ]:
df_2010.to_csv('censo_2010.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2011
df_2011 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2011]

In [ ]:
df_2011.to_csv('censo_2011.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2012
df_2012 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2012]

In [ ]:
df_2012.to_csv('censo_2012.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2013
df_2013 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2013]

In [ ]:
df_2013.to_csv('censo_2013.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2014
df_2014 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2014]

In [ ]:
df_2014.to_csv('censo_2014.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2015
df_2015 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2015]

In [ ]:
df_2015.to_csv('censo_2015.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2016
df_2016 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2016]

In [ ]:
df_2016.to_csv('censo_2016.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2017
df_2017 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2017]

In [ ]:
df_2017.to_csv('censo_2017.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2018
df_2018 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2018]

In [ ]:
df_2018.to_csv('censo_2018.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
df_2019 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2019]

In [ ]:
#CENSO 2019
df_2019.to_csv('censo_2019.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2020
df_2020 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2020]

In [ ]:
df_2020.to_csv('censo_2020.csv', sep=';', encoding=cod_23, index=False)

In [ ]:
#CENSO 2021
df_2021 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2021]

In [ ]:
df_2021.to_csv('censo_2021.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2022
df_2022 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2022]

In [ ]:
df_2022.to_csv('censo_2022.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2023
df_2023 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2023]

In [ ]:
df_2023.to_csv('censo_2023.csv', sep=";", encoding=cod_23, index=False)

In [ ]:
#CENSO 2024
df_2024 = df_full_order[df_full_order['NU_ANO_CENSO'] == 2024]

In [ ]:
df_2024.to_csv('censo_2024.csv', sep=";", encoding=cod_23, index=False)

In [689]:
#ALLCENSOS
df_full_order.to_csv('censo_geral.csv', sep=";", encoding=cod_23, index=False)